# EDA: fraud detection
### Fraud Detection in Electricity and Gas Consumption Challenge from Zindi
### Detect clients commiting fraud in Tunisia regarding their electricity and gas consumption based on their billing history 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt

from eda_tables import explore
import eda_viz as ev 

import data first check

In [ ]:
#import data
client_raw = pd.read_csv('data/client_train.csv')
invoice_raw = pd.read_csv('data/invoice_train.csv')

## Data Exploration 

In [ ]:
#first info including missing values and duplicates
explore(invoice_raw)

In [ ]:
#first info including missing values and duplicates
explore(client_raw)

In [ ]:
# renaming some columns
client_raw = client_raw.rename(columns={'disrict': 'district'})

invoice_raw = invoice_raw.rename(columns={'consommation_level_1': 'CL_1', 'consommation_level_2': 'CL_2',
                                          'consommation_level_3': 'CL_3', 'consommation_level_4': 'CL_4'})

In [ ]:
#client value counts
client_cat = client_raw[['district', 'client_catg', 'region', 'target']]
ev.categorical_value_counts(client_cat)

In [ ]:
# Distribution of clients per district and region
for col in ['district','region']:
    region = client_raw.groupby([col])['client_id'].count()
    plt.bar(x=region.index, height=region.values)
    plt.title('Distribution across ' + col +'s')
    plt.show()

In [ ]:
#Distribution of frauds to non frauds
fraud = client_raw.groupby(['target'])['client_id'].count()
fraud_percentage = (fraud / fraud.sum()) * 100
plt.bar(x=fraud.index, height=fraud.values, tick_label=[0, 1])
plt.title('Fraud Distribution')

for index, value in enumerate(fraud.values):
    percentage = fraud_percentage[index]
    plt.text(index, value + 10, f'{percentage:.2f}%', ha='center', va='top', fontweight='bold')

plt.show()

In [ ]:
#Distribution of frauds per district and region
for col in ['district','region']:
   
    total_clients = client_raw.groupby(col)['client_id'].count()
    total_frauds = client_raw.groupby(col)['target'].sum()
    relative_frauds = (total_frauds / total_clients) * 100
    
    #plt.figure(figsize=(10, 5))
    plt.bar(relative_frauds.index, relative_frauds.values)
    plt.title(f'Relative frauds per {col}')
    plt.xlabel(col.capitalize())
    plt.ylabel('Percentage of frauds')
    plt.show()

In [ ]:
#invoice value counts
invoice_cat = invoice_raw[['tarif_type', 'counter_statue', 'counter_code', 'reading_remarque',
                           'counter_coefficient','counter_type']]

ev.categorical_value_counts(invoice_cat)

In [ ]:
CLs = invoice_raw[['CL_1','CL_2','CL_3','CL_4']]
ev.plot_distributions(CLs)

Remove duplicates, NaNs, outliers

In [ ]:
#no NaNs in the data

In [ ]:
# remove duplicates
invoice_cleaned = invoice_raw.drop_duplicates()

In [ ]:
#remove extreme outliers (1% and 99% quantile, instead of e.g. z-score, due to extreme skewness)
for col in ['CL_1', 'CL_2', 'CL_3', 'CL_4']:
    lower_bound = invoice_cleaned[col].quantile(0.01)
    upper_bound = invoice_cleaned[col].quantile(0.99)
    invoice_filtered = invoice_cleaned[(invoice_cleaned[col] >= lower_bound) & (invoice_cleaned[col] <= upper_bound)]


### feature engineering

In [ ]:
# create additional values
CLs = ['CL_1','CL_2','CL_3','CL_4']

for CL in CLs: 
    CL_diff = f'diff_{CL}'
    inc_diff = f'inc_{CL}'
    rel_diff = f'rel_diff_{CL}'
    large_diff = f'large_diff_{CL}'
    invoice_filtered[CL] = invoice_filtered[CL].replace(0, 1e-9)  # Replaces 0 with a tiny number

    invoice_filtered[CL_diff] = invoice_filtered.groupby('client_id')[CL].diff().fillna(0)  
    invoice_filtered[inc_diff] = (invoice_filtered.groupby('client_id')[CL].diff().fillna(0) > 0).astype(int)
    invoice_filtered[rel_diff] = invoice_filtered.groupby('client_id')[CL].pct_change().fillna(0) 
    invoice_filtered[large_diff] = (invoice_filtered.groupby('client_id')[CL].pct_change().fillna(0) > invoice_filtered[CL].quantile(0.75) ).astype(int) 

print(invoice_filtered.head(10))


In [ ]:
def majority_or_single(series):
    if len(series) == 1:
        return series.iloc[0]  # Take the single value if only one invoice
    counts = series.value_counts()
    if len(counts) > 1 and counts.iloc[0] == counts.iloc[1]:
        return 1  # If there's a tie, take True (1)
    return counts.idxmax()  # Otherwise, take the most frequent value (mode)

# Define custom function to calculate the percentage of 1s 
def percentage_of_ones(series):
    return (series.sum() / len(series)) 


# Aggregate diff_features
diff_features_bool = invoice_filtered.groupby('client_id')[['inc_CL_1','inc_CL_2','inc_CL_3','inc_CL_4',
                                                            'large_diff_CL_1','large_diff_CL_2','large_diff_CL_3','large_diff_CL_4'
                                                            ]].agg(majority_or_single).reset_index()
diff_features_rel = invoice_filtered.groupby('client_id')[['inc_CL_1','inc_CL_2','inc_CL_3','inc_CL_4',
                                                           'large_diff_CL_1','large_diff_CL_2','large_diff_CL_3','large_diff_CL_4'
                                                           ]].agg(percentage_of_ones).reset_index()
# Ensure the values are integers
diff_features_bool[['inc_CL_1','inc_CL_2','inc_CL_3','inc_CL_4',
               'large_diff_CL_1','large_diff_CL_2','large_diff_CL_3','large_diff_CL_4'
               ]] = diff_features_bool[[
                   'inc_CL_1','inc_CL_2','inc_CL_3','inc_CL_4',
                   'large_diff_CL_1','large_diff_CL_2','large_diff_CL_3','large_diff_CL_4'
               ]].astype(int)

# Add suffix to every column name except client_id
diff_features_rel = diff_features_rel.rename(columns=lambda col_name: col_name if col_name == 'client_id' else col_name + '_rel')

In [ ]:
# Group by column 'client_id' and calculate the mean of consommation_levels
mean_result = invoice_filtered.groupby('client_id')[['CL_1','CL_2','CL_3','CL_4']].mean().reset_index() 
# Count the number of elements in each group 
count_result = invoice_filtered.groupby('client_id')[['CL_1','CL_2','CL_3','CL_4']].size().reset_index(name='no_invoices') 

# Merge the mean and count results 
merged_invoice_1 = pd.merge(mean_result, count_result,  on='client_id')  
merged_invoice_2 = pd.merge(merged_invoice_1 , diff_features_bool,  on='client_id')  
merged_invoice = pd.merge(merged_invoice_2, diff_features_rel,  on='client_id')  

In [ ]:
client = client_raw # nothing to clean, filter, etc in client data

In [ ]:
#merge dfs
df = client.merge(merged_invoice, how='left', on= 'client_id')

df.shape[1] == merged_invoice.shape[1] + client.shape[1]-1 # ncol equal? Yes!

In [ ]:
# create date columns
df['creation_date'] = pd.to_datetime(df['creation_date'])
df['day'] = df['creation_date'].dt.day.apply(lambda x: f'{x:02d}')
df['month'] = df['creation_date'].dt.month.apply(lambda x: f'{x:02d}')
df['year'] = df['creation_date'].dt.year
df['date_int'] = df[['year', 'month', 'day']].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [ ]:
# final check
explore(df)
# missing values, no of unique client ids in df 'client' > no of unique client ids in df 'merged_invoice'
# --> remove rows with na
df = df.dropna()

In [ ]:
# save df as csv

df.to_csv('data/merged_train.csv', index=False)